In [19]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

In [16]:
# Function for comparing different models
def score_dataset(x_train, x_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(x_train, y_train)
    preds = model.predict(x_valid)
    return mean_absolute_error(y_valid, preds)

In [9]:
# Read data
data = pd.read_csv("melbourne data.csv")

# target object
y = data.Price

# predictor
# get data without price column
predictor = data.drop(['Price'], axis=1)
# get numerical data only
x = predictor.select_dtypes(exclude=['object'])

In [11]:
# Divide data into training and validation data
x_train, x_valid, y_train, y_valid = train_test_split(x, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [18]:
# Missing values
# 1.Drop columns
# get names of columns with missing values
cols_with_missing = [col for col in x_train.columns
                     if x_train[col].isnull().any()]
# Drop columns in training and validation data
reduced_x_train = x_train.drop(cols_with_missing, axis=1)
reduced_x_valid = x_valid.drop(cols_with_missing, axis=1)

# get mae
print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_x_train, reduced_x_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


In [24]:
# Missing values
# 2. Imputation
my_imputer = SimpleImputer()
imputed_x_train = pd.DataFrame(my_imputer.fit_transform(x_train))
imputed_x_valid = pd.DataFrame(my_imputer.transform(x_valid))

# Imputation removed column names; put them back
imputed_x_train.columns = x_train.columns
imputed_x_valid.columns = x_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_x_train, imputed_x_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
178166.46269899711


In [28]:
# Make copy to avoid changing original data (when imputing)
x_train_plus = x_train.copy()
x_valid_plus = x_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    x_train_plus[col + ' was missing'] = x_train_plus[col].isnull()
    x_valid_plus[col + ' was missing'] = x_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_x_train_plus = pd.DataFrame(my_imputer.fit_transform(x_train_plus))
imputed_x_valid_plus = pd.DataFrame(my_imputer.transform(x_valid_plus))

# Imputation removed column names; put them back
imputed_x_train_plus.columns = x_train_plus.columns
imputed_x_valid_plus.columns = x_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_x_train_plus, imputed_x_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


In [31]:
imputed_x_train_plus.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount,Car was missing,BuildingArea was missing,YearBuilt was missing
0,1.0,5.0,3182.0,1.0,1.0,1.0,0.0,153.764119,1940.000000,-37.85984,144.9867,13240.0,0.0,1.0,0.0
1,2.0,8.0,3016.0,2.0,2.0,1.0,193.0,153.764119,1964.839866,-37.85800,144.9005,6380.0,0.0,1.0,1.0
2,3.0,12.6,3020.0,3.0,1.0,1.0,555.0,153.764119,1964.839866,-37.79880,144.8220,3755.0,0.0,1.0,1.0
3,3.0,13.0,3046.0,3.0,1.0,1.0,265.0,153.764119,1995.000000,-37.70830,144.9158,8870.0,0.0,1.0,0.0
4,3.0,13.3,3020.0,3.0,1.0,2.0,673.0,673.000000,1970.000000,-37.76230,144.8272,4217.0,0.0,0.0,0.0
